In [1]:
import pandas as pd
import altair as alt

In [2]:
data_prefix = 'tennis_slam_pointbypoint/'

In [3]:
player = 'Roger Federer'
tourney = 'wimbledon'

In [4]:
year_speeds = []
for year in ['2014', '2015', '2016', '2017', '2018', '2019', '2021']:
    matches = pd.read_csv(data_prefix + year + '-' + tourney + '-matches.csv')
    matches['year'] = matches['match_id'].str[:4]
    points = pd.read_csv(data_prefix + year + '-' + tourney + '-points.csv')
    
    ## add match metadata to points rows
    mpoints = pd.merge(points, matches).fillna(0)
    
    mpoints.astype({'PointServer': 'int',
                    'ServeNumber': 'int',
                    'Speed_KMH': 'int'
               }).dtypes
    
    ## skip  bad data with serve speed of zero
    mpoints = mpoints.loc[mpoints['Speed_KMH'] != 0]
    
    ## points with target player serving
    serving1 = mpoints.loc[(mpoints['PointServer'] == 1) & (mpoints['player1'] == player)]
    serving2 = mpoints.loc[(mpoints['PointServer'] == 2) & (mpoints['player2'] == player)]
    svpoints = pd.concat([serving1, serving2])
    
    ## first serve points from above
    fsv_points = svpoints.loc[svpoints['ServeNumber'] == 1]
    
    ## Rounds 1, 2, and 3 from above (round number is 3rd-to-last char in match_id)
    first_three_rounds = fsv_points.loc[fsv_points['match_id'].str[-3].isin(['1', '2', '3'])]
    
    ## reduce data to year and serve-speed only
    year_speeds += first_three_rounds[['year', 'Speed_KMH']].values.tolist()

/var/folders/7v/tz09g0015bnfg4mm4rjllkxr0000gn/T/ipykernel_4561/2301195140.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  points = pd.read_csv(data_prefix + year + '-' + tourney + '-points.csv')


In [5]:
df = pd.DataFrame(year_speeds, columns=['Year', 'KM/H'])
df.astype({'KM/H': 'float', 
            }).dtypes

Year     object
KM/H    float64
dtype: object

In [6]:
serve_plot = alt.Chart(df).mark_boxplot().encode(
    alt.X('Year:N',
          ),
    alt.Y('KM/H:Q',
          scale=alt.Scale(domain=(120, 220))
          )
).properties(
    width=400,
    height=350
)

In [7]:
serve_plot

alt.Chart(...)

In [8]:
out_path = 'output/' + tourney + '_' + player.replace(' ', '_') + '_first_week_first_serves.html'
serve_plot.save(out_path)